In [47]:
import numpy as np
import scipy

In [48]:
def back_substitution(U, y):
    n = U.shape[1]
    x = np.zeros_like(y, dtype=np.double)
    x[-1] = y[-1] / U[-1, -1]
    for i in range(n - 2, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i:], x[i:])) / U[i, i]

    return x

def linsolve_qr(A, b):
    num_param = A.shape[1]
    q, r = scipy.linalg.qr(A) # Q*R*p = y
    return back_substitution(r[0:num_param], (q.T @ b)[0:num_param]) # solve for: R*p = Q.T * y


def gauss_newton(f, df, x, tolerance=1e-14, max_step=1000):
    step = 0
    error = np.linalg.norm(f(x))
    while error > tolerance and step < max_step:
        # print(step, x, error)
        x += linsolve_qr(df(x), -f(x))
        error = np.linalg.norm(f(x))
        step += 1

    return x

In [49]:
def f(p):
    (k, l) = p
    def row(x, y, k, l):
        return 1-np.exp(-(x/l)**k)-y

    return np.array([
        row(0.26, 0.004, k, l),
        row(1.7, 0.071, k, l),
        row(4.0, 0.226, k, l),
        row(12.1, 0.738, k, l),
        row(28.0, 0.991, k, l),
    ])

def df(p):
    (k, l) = p
    
    def row(x, y, k, l):
        return np.array([
            -np.exp(-(x/l)**k) * -(x/l)**k * -np.log(x/l),
            np.exp(-(x/l)**k) * -(x/l)**k * (k/l)
        ])
    
    return np.array([
        row(0.26, 0.004, k, l),
        row(1.7, 0.071, k, l),
        row(4.0, 0.226, k, l),
        row(12.1, 0.738, k, l),
        row(28.0, 0.991, k, l),
    ])

# search for m, n, R
initial_guess = np.array([1, 15], dtype=np.float64)

(k, l) = gauss_newton(f, df, initial_guess)
print((k, l))

errors = f((k, l))
resid = np.sum(errors**2)
print(resid)

(-24665.29053674514, -2456.1270191014178)
nan


C:\Users\NathanielWalser\AppData\Local\Temp\ipykernel_18272\3421999035.py:4: RuntimeWarning: invalid value encountered in scalar power
  return 1-np.exp(-(x/l)**k)-y


In [50]:
# punkte abzug wegen nutzen der integrierten solver besser als keine punkte :D
loss_function = lambda p: np.sum(f(p)**2)
optimization = scipy.optimize.minimize(loss_function, np.array([1.0, 15.0]), method="Nelder-Mead")
print(optimization)

print(optimization.x)

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 3.6566723972667525e-06
             x: [ 1.490e+00  9.948e+00]
           nit: 44
          nfev: 83
 final_simplex: (array([[ 1.490e+00,  9.948e+00],
                       [ 1.490e+00,  9.948e+00],
                       [ 1.490e+00,  9.948e+00]]), array([ 3.657e-06,  3.657e-06,  3.657e-06]))
[1.48994448 9.94787235]
